<a href="https://colab.research.google.com/github/rennyatwork/CegepSteFoy_IA/blob/main/08_deep_learning/examen_final_q_11_RadioEncoder_Incomplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nous voulons uitiliser l'IA génmérative pour simuler diverses formed de boutons de radio (stop, pause, play, avancer, reculer)

1. Importation des packages utiles

In [2]:
import numpy as np
from numpy import asarray
from numpy import unique
from numpy import argmax
from numpy import sqrt
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

from tensorflow.keras import layers, models, datasets, callbacks, metrics
import tensorflow.keras.backend as K

2. Importer les données brutes

NOTE: on doit d'abord télécharger le fichier RadioCommandsData.csv sur le site web du cours dans la section "Contenu et activités / Séance en direct 6 - CNN et vision informatique

In [3]:
df = read_csv('/content/drive/MyDrive/Colab Datasets/RadioCommandsData.csv')
print(len(df))
df.head(10)

FileNotFoundError: ignored

3. Reformatter les X en images au format 8x8x1 (teintes de gris)

In [ ]:
X_flat = np.array(df)
X_flat = X_flat[:,6:]
X_flat = X_flat.astype(np.float32)

X_complete = np.reshape(X_flat,(330,8,8,1))
print(X_complete.shape)

4. Obtenir les labels (y) des images

NOTE: 0: pause, 1: stop, 2: play, 3: backward, 4: forward

In [ ]:
y_complete = np.array(df)[:,1]

y_complete[:10]

5. Diviser les données en train et test data

In [ ]:
X_train_full, X_test, y_train_full, y_test = train_test_split(X_complete, y_complete, test_size=0.33)
print(X_train_full.shape, X_test.shape, y_train_full.shape, y_test.shape)

6. Conserver une portion des données en validation

In [ ]:
X_train, y_train = X_train_full[:-50], y_train_full[:-50]
X_valid, y_valid = X_train_full[-50:], y_train_full[-50:]
print(X_train_full.shape, X_test.shape,X_train.shape,X_valid.shape, y_train_full.shape, y_test.shape,y_train.shape,y_valid.shape)

7. Modifier la "shape" des données pour se conformer aux entrées attendus de Keras et storer la shape des images d'input dans une variable nommée in_shape (pour le première couche d'entrée)

In [ ]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))
X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1], X_valid.shape[2], 1))

in_shape = X_train.shape[1:]
print(in_shape)

8. Visualiser quelques images

In [ ]:
from matplotlib import pyplot
for i in range(9):
  pyplot.subplot(330 + 1 + i)
  pyplot.imshow(X_train[i], cmap=pyplot.get_cmap('gray'))
pyplot.show()

9. Storer le nombre de clasees possibles (5 dans ce cas-ci, car 0, 1, 2, 3, 4) dans la variable n_classes

In [ ]:
n_classes = len(unique(y_train))
print(n_classes)

10. Bâtir l'IA générative ... rendu ici ...

In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.models import Model

# Define the VAE architecture
latent_dim = 32  # Size of the latent space

# Encoder
encoder_inputs = Input(shape=in_shape)
x = Flatten()(encoder_inputs)
x = Dense(128, activation='relu')(x)
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

# Sampling layer
from tensorflow.keras.layers import Lambda
from tensorflow.keras import backend as K

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

# Decoder
decoder_inputs = Input(shape=(latent_dim,))
x = Dense(128, activation='relu')(decoder_inputs)
x = Dense(64, activation='relu')(x)
decoder_outputs = Dense(np.prod(in_shape), activation='sigmoid')(x)
decoder_outputs = Reshape(in_shape)(decoder_outputs)

# Define the VAE model
encoder = Model(encoder_inputs, z)
decoder = Model(decoder_inputs, decoder_outputs)

vae_outputs = decoder(encoder(encoder_inputs))
vae = Model(encoder_inputs, vae_outputs)

# Define the VAE loss
from tensorflow.keras.losses import mse, binary_crossentropy

reconstruction_loss = mse(encoder_inputs, vae_outputs)
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)

vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

# Train the VAE
vae.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_valid, X_valid))

# Generate some images using the trained VAE
decoded_imgs = vae.predict(X_test)
